In [2]:
%load_ext autoreload
%autoreload 2

from fastai.text.all import *
from fastai.vision.all import *
import pandas as pd
import torch
from tqdm.notebook import tqdm

from utils import get_dls

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

# # tensorflow RNG
# tf.random.set_seed(seed)

In [4]:
path = Path("/mnt/nas/databases/Tobacco800/unziped/page_imgs/raw")

In [5]:
df = pd.read_csv('/mnt/nas/databases/Tobacco800/unziped/train.csv', delimiter=';',  usecols=['binder','docid','class', 'text'])

In [6]:
print(df.shape)
df.dropna(inplace=True)
print(df.shape)

(1031, 4)
(1019, 4)


In [7]:
df['split'] = 'train';print(df.shape); df.head()

(1019, 5)


,binder,docid,class,text,split
0,Tobacco800,aah97e00-page02_1,FirstPage,"Dr. M.A. Manzelli, PHILIP MORRIS INC., Research Center, P.O. Box 3 D Richmond, Va. 23206 U.S.A. Dear Art, Please find enclosed a proposed program for your visit in Europe. As you will see, this program contains two major points: a) A two-days meeting in Neuchätel with all people in Europe involved in infestation control (proposed program included). b) Visit of all PM Europe production centers with the purpose of: - training people in Charge of the insect indenti-fication - inspection of the sanitary conditions One of our main problems is the insect Identification and ...",train
1,Tobacco800,aah97e00-page02_2,NextPage,"- 2 - Please let me krow if you have any changes for the proposed program. Looking forward to seeing you again, cc: Messrs. B. Kuhn E. Stoop Encl.: mentioned 1000035625",train
2,Tobacco800,aam09c00,FirstPage,"I NOIJ-04-97 13 = 25 FROM = I D : PAGE 10/17 Wachtell, Lipton, Rosen & Katz MARTIN L1PTON HERBERT M. WaCHYCLL THEODORE GEWSRTZ »ERNA*» W, NUSSBAUM RICHARD O- HATCHER DOUGLAS Sr LieBKAfSKV STEVEN M. SAR NA CMARt.CS T. MEPCRR'GK CHAIN J. FOPTCANG pBTRR C. 1AMCLLDS MICHAEL w. SCHWÄRT* Al.LAR A- MARTIN BARRV A. BRrCR IAWREHCE □. PEDQrWITZ ROBERT 0. MAZUR PAUL VIZGARRÖMSCK JR. PCTCR G- HCIM hAROlD s. «OViKOFF DAVID M. EINHORN KENNET« B. FQRREST METER C. KOPLOW THEODORE M- MIRVIS EDWARD D. MERL1HT RICHARD D. FEINTUCH DANIEL A. NEPP ERIC M. ROTR WAHREN R. STERN ANDREW R. BROWN ST EIN MICHAEL H...",train
3,Tobacco800,aao54e00_1,FirstPage,"i PHILIP .MORRIS INCORPORATED 120 PARK AVENUE, N EW YORK. N. Y. 10017 John T.-Landhy SENIOR VICC PRESIDENT _ , r . n n - . oirector of Marketing February 15, 19 83 Dear Ms. Ober: Your letter of February 3 was received by this Office yesterday. We attempted to reach you by telephone iinmediately but were unable to find a listed number in New Haven. I had hoped to arrange an appointment for one of our executives to discuss with you and your concerned ...",train
4,Tobacco800,aao54e00_2,NextPage,"In the meantime, I hope you and your friends are continuing to enjoy your Marlboro Lights ciga-rettes. We don't think you can find a better"" cigarette on the market. ■Thank you. Ms. Angela Ober 221 Farnham Avenue New Haven, Connecticut 06515",train


In [8]:
df['split'][-200:] = 'valid'; df.iloc[-202:-198];

In [9]:
df

,binder,docid,class,text,split
0,Tobacco800,aah97e00-page02_1,FirstPage,"Dr. M.A. Manzelli, PHILIP MORRIS INC., Research Center, P.O. Box 3 D Richmond, Va. 23206 U.S.A. Dear Art, Please find enclosed a proposed program for your visit in Europe. As you will see, this program contains two major points: a) A two-days meeting in Neuchätel with all people in Europe involved in infestation control (proposed program included). b) Visit of all PM Europe production centers with the purpose of: - training people in Charge of the insect indenti-fication - inspection of the sanitary conditions One of our main problems is the insect Identification and ...",train
1,Tobacco800,aah97e00-page02_2,NextPage,"- 2 - Please let me krow if you have any changes for the proposed program. Looking forward to seeing you again, cc: Messrs. B. Kuhn E. Stoop Encl.: mentioned 1000035625",train
2,Tobacco800,aam09c00,FirstPage,"I NOIJ-04-97 13 = 25 FROM = I D : PAGE 10/17 Wachtell, Lipton, Rosen & Katz MARTIN L1PTON HERBERT M. WaCHYCLL THEODORE GEWSRTZ »ERNA*» W, NUSSBAUM RICHARD O- HATCHER DOUGLAS Sr LieBKAfSKV STEVEN M. SAR NA CMARt.CS T. MEPCRR'GK CHAIN J. FOPTCANG pBTRR C. 1AMCLLDS MICHAEL w. SCHWÄRT* Al.LAR A- MARTIN BARRV A. BRrCR IAWREHCE □. PEDQrWITZ ROBERT 0. MAZUR PAUL VIZGARRÖMSCK JR. PCTCR G- HCIM hAROlD s. «OViKOFF DAVID M. EINHORN KENNET« B. FQRREST METER C. KOPLOW THEODORE M- MIRVIS EDWARD D. MERL1HT RICHARD D. FEINTUCH DANIEL A. NEPP ERIC M. ROTR WAHREN R. STERN ANDREW R. BROWN ST EIN MICHAEL H...",train
3,Tobacco800,aao54e00_1,FirstPage,"i PHILIP .MORRIS INCORPORATED 120 PARK AVENUE, N EW YORK. N. Y. 10017 John T.-Landhy SENIOR VICC PRESIDENT _ , r . n n - . oirector of Marketing February 15, 19 83 Dear Ms. Ober: Your letter of February 3 was received by this Office yesterday. We attempted to reach you by telephone iinmediately but were unable to find a listed number in New Haven. I had hoped to arrange an appointment for one of our executives to discuss with you and your concerned ...",train
4,Tobacco800,aao54e00_2,NextPage,"In the meantime, I hope you and your friends are continuing to enjoy your Marlboro Lights ciga-rettes. We don't think you can find a better"" cigarette on the market. ■Thank you. Ms. Angela Ober 221 Farnham Avenue New Haven, Connecticut 06515",train
...,...,...,...,...,...
1026,Tobacco800,thl51a00-page02_2,NextPage,"«AR 18 »Sg 14t99 FROM B/W LAU DEPT PAGE.Q03 P*fl® t Maroh 18, 1992 Letter fco K. fii yeeterday 1» which American would represent that it does not m)w any itlai of the wcxt STRIKs «ad fall mall producta in the United States, inoiuding tu® U.S. duty free narbet# fot re-oacpott to »ttd resale in any country where Brom * wlllieason# or a mamber of the b.a.t. Group of 0Oa&anies, has registered theso tradomarks# and that any sudü aale by any parson woUld be oonaidered an infringement of Brown & Williamson*® tredemark rights. 4 3* Pr...",valid
1027,Tobacco800,tji44a00,FirstPage,"MOV 2 11994 Si iook.J Iakdyü. Bacon November 18, 1994 FEDERAL EXPRESS TO: ETS/IAQ Current Developments Report Distribution Enclosed you will find a copy of the most recent issue of the ETS/XACl Current Developments Report. If you have questions or information rega.rd.ing items contained in this Report, please feel free to contact either of us at (816) 4*74-6550. Sincerely MWC : KAM : lew Enclosure cc: Report: Team 88325694",valid
1028,Tobacco800,tjr72f00-page02_1,FirstPage,"THE TOBACCO INSTITUTE, inc. ■1776 K STREET, NORTHWEST! WASHINGTON, D.C. 2000b i 202.'296-8434 HORACE R. KORNEGAY President May 13, 1975 Mr. Colin Stokes Chairman of the Board of Directors R. J. Reynolds Industries, Inc. 401 North Main Street Winston-Salem, North Carolina 27102 Dear Colin: The terms on which your and other Companies agreed to participate in the establishment and Operation of the Tobacco Institute Testing Laboratory (T.I.T.L.) were set förth in the Institute letter of November 1, 1966, which was accepted by each participating Company. Based on recent discuss

In [10]:
df_test = pd.read_csv('/mnt/nas/databases/Tobacco800/unziped/test.csv', delimiter=';',  usecols=['binder','docid','class'])

In [11]:
df_test['split'] = 'test'

In [12]:
df = pd.concat([df, df_test], axis=0); df.reset_index(drop=True, inplace=True); df

,binder,docid,class,text,split
0,Tobacco800,aah97e00-page02_1,FirstPage,"Dr. M.A. Manzelli, PHILIP MORRIS INC., Research Center, P.O. Box 3 D Richmond, Va. 23206 U.S.A. Dear Art, Please find enclosed a proposed program for your visit in Europe. As you will see, this program contains two major points: a) A two-days meeting in Neuchätel with all people in Europe involved in infestation control (proposed program included). b) Visit of all PM Europe production centers with the purpose of: - training people in Charge of the insect indenti-fication - inspection of the sanitary conditions One of our main problems is the insect Identification and ...",train
1,Tobacco800,aah97e00-page02_2,NextPage,"- 2 - Please let me krow if you have any changes for the proposed program. Looking forward to seeing you again, cc: Messrs. B. Kuhn E. Stoop Encl.: mentioned 1000035625",train
2,Tobacco800,aam09c00,FirstPage,"I NOIJ-04-97 13 = 25 FROM = I D : PAGE 10/17 Wachtell, Lipton, Rosen & Katz MARTIN L1PTON HERBERT M. WaCHYCLL THEODORE GEWSRTZ »ERNA*» W, NUSSBAUM RICHARD O- HATCHER DOUGLAS Sr LieBKAfSKV STEVEN M. SAR NA CMARt.CS T. MEPCRR'GK CHAIN J. FOPTCANG pBTRR C. 1AMCLLDS MICHAEL w. SCHWÄRT* Al.LAR A- MARTIN BARRV A. BRrCR IAWREHCE □. PEDQrWITZ ROBERT 0. MAZUR PAUL VIZGARRÖMSCK JR. PCTCR G- HCIM hAROlD s. «OViKOFF DAVID M. EINHORN KENNET« B. FQRREST METER C. KOPLOW THEODORE M- MIRVIS EDWARD D. MERL1HT RICHARD D. FEINTUCH DANIEL A. NEPP ERIC M. ROTR WAHREN R. STERN ANDREW R. BROWN ST EIN MICHAEL H...",train
3,Tobacco800,aao54e00_1,FirstPage,"i PHILIP .MORRIS INCORPORATED 120 PARK AVENUE, N EW YORK. N. Y. 10017 John T.-Landhy SENIOR VICC PRESIDENT _ , r . n n - . oirector of Marketing February 15, 19 83 Dear Ms. Ober: Your letter of February 3 was received by this Office yesterday. We attempted to reach you by telephone iinmediately but were unable to find a listed number in New Haven. I had hoped to arrange an appointment for one of our executives to discuss with you and your concerned ...",train
4,Tobacco800,aao54e00_2,NextPage,"In the meantime, I hope you and your friends are continuing to enjoy your Marlboro Lights ciga-rettes. We don't think you can find a better"" cigarette on the market. ■Thank you. Ms. Angela Ober 221 Farnham Avenue New Haven, Connecticut 06515",train
...,...,...,...,...,...
1273,Tobacco800,zrz94a00-page02_2,NextPage,NaN,test
1274,Tobacco800,zss86d00,FirstPage,NaN,test
1275,Tobacco800,ztz52d00-page02_1,FirstPage,NaN,test
1276,Tobacco800,ztz52d00-page02_2,NextPage,NaN,test


In [13]:
def splitter(df):
    train = df[df['split']=='train'].index.tolist()
    valid = df[df['split']=='valid'].index.tolist()
    test = df[df['split']=='test'].index.tolist()
    return train,valid, test

In [14]:
def get_x(r): return path/f'{r["docid"]}.tif'
def get_y(r): return r['class']

In [15]:
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_x=get_x,
                   get_y=get_y,
                   splitter=splitter,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=224, min_scale=0.9,
                                               do_flip=False, max_rotate=0,
                                               max_warp=0),
                               Normalize.from_stats(*imagenet_stats)])

In [16]:
dls = dblock.dataloaders(df, bs=64)

In [17]:
#test_dl = dls.test_dl(test_items, with_labels=True)

In [18]:
class GetActs(HookCallback):
    def __init__(self, modules=None, remove_end=True, detach=True, cpu=True):
        super().__init__(modules, None, remove_end, True, detach, cpu)
        self.acts = L()
    def hook(self, m, i, o): return o
    def after_pred(self): self.acts += self.hooks.stored
    def before_fit(self):
        super().before_fit()
        self.acts = L()

In [19]:
learn = cnn_learner(dls, resnet50, loss_func=CrossEntropyLossFlat())

In [20]:
learn.add_cb(GetActs([learn.model[1][1]]))

In [21]:
learn.load("best_image_no_weights_224")

In [22]:
learn.validate()

(#1) [0.762245774269104]

In [23]:
valid_preds = learn.get_acts.acts

In [24]:
valid_preds = torch.cat(list(valid_preds)); valid_preds.shape

torch.Size([200, 4096])

In [25]:
sample = list(dls.valid.items)

In [26]:
#dls.valid.items

In [26]:
assert len(valid_preds) == len(dls.valid_ds)

In [28]:
idx = 0
for index, row in dls.valid.items.iterrows():
    filename = Path("./activations/img/val/")/row['class']/row['docid']
    filename.parent.mkdir(parents=True, exist_ok=True)
    torch.save(valid_preds[idx].clone(), filename.as_posix() + ".pt")
    print(f"Saving example {idx+1}", end='\r', flush=True)
    idx += 1

In [29]:
dls.train.shuffle = False

In [30]:
dls.train.get_idxs()[:5]

[0, 1, 2, 3, 4]

In [31]:
learn.get_preds(0)

(TensorImage([[0.9395, 0.0605],
         [0.0492, 0.9508],
         [0.9904, 0.0096],
         ...,
         [0.9767, 0.0233],
         [0.8923, 0.1077],
         [0.9785, 0.0215]]),
 TensorCategory([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
         0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
         0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
         0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
         1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
         0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,

In [32]:
train_preds = learn.get_acts.acts

In [33]:
train_preds = torch.cat(list(train_preds)); train_preds.shape

torch.Size([819, 4096])

In [34]:
assert len(train_preds) == len(dls.train_ds)

In [35]:
#dls.train.items[dls.train.items['docid']=='pkc56d00']
dls.train.items.shape#.iloc[-1]

(819, 5)

In [36]:
idx = 0
for index, row in dls.train.items.iterrows():
    filename = Path("./activations/img/train/")/row['class']/row['docid']
    filename.parent.mkdir(parents=True, exist_ok=True)
    torch.save(train_preds[idx].clone(), filename.as_posix() + ".pt")
    print(f"Saving example {idx+1}", end='\r', flush=True)
    idx += 1

In [37]:
learn.get_preds(ds_idx=2)

(TensorImage([[4.2778e-01, 5.7222e-01],
         [1.0870e-01, 8.9130e-01],
         [2.8373e-02, 9.7163e-01],
         [5.7938e-02, 9.4206e-01],
         [4.9627e-02, 9.5037e-01],
         [2.4094e-02, 9.7591e-01],
         [3.9858e-02, 9.6014e-01],
         [2.2283e-02, 9.7772e-01],
         [9.6245e-02, 9.0375e-01],
         [9.5487e-01, 4.5129e-02],
         [9.7871e-01, 2.1290e-02],
         [9.4004e-01, 5.9956e-02],
         [7.9603e-01, 2.0398e-01],
         [7.1778e-01, 2.8222e-01],
         [9.5444e-01, 4.5557e-02],
         [8.7475e-01, 1.2525e-01],
         [9.4711e-01, 5.2893e-02],
         [9.3032e-01, 6.9681e-02],
         [8.0979e-01, 1.9021e-01],
         [8.1617e-01, 1.8383e-01],
         [8.1592e-01, 1.8408e-01],
         [8.3721e-01, 1.6279e-01],
         [9.8391e-01, 1.6093e-02],
         [7.5690e-01, 2.4310e-01],
         [5.7424e-01, 4.2576e-01],
         [3.8108e-01, 6.1892e-01],
         [9.8129e-01, 1.8713e-02],
         [6.4655e-01, 3.5345e-01],
         [6.994

In [38]:
test_preds = learn.get_acts.acts

In [39]:
test_preds = torch.cat(list(test_preds)); test_preds.shape

torch.Size([259, 4096])

In [40]:
#dls[2].items

In [41]:
assert len(test_preds) == len(dls[2].items)

In [42]:
idx = 0
for index, row in dls[2].items.iterrows():
    filename = Path("./activations/img/test/")/row['class']/row['docid']
    filename.parent.mkdir(parents=True, exist_ok=True)
    torch.save(test_preds[idx].clone(), filename.as_posix() + ".pt")
    print(f"Saving example {idx+1}", end='\r', flush=True)
    idx += 1

In [43]:
dls[2].items

,binder,docid,class,text,split
1019,Tobacco800,tkj51f00_1,FirstPage,NaN,test
1020,Tobacco800,tkj51f00_2,NextPage,NaN,test
1021,Tobacco800,tkj51f00_3,NextPage,NaN,test
1022,Tobacco800,tkj51f00_4,NextPage,NaN,test
1023,Tobacco800,tkj51f00_5,NextPage,NaN,test
...,...,...,...,...,...
1273,Tobacco800,zrz94a00-page02_2,NextPage,NaN,test
1274,Tobacco800,zss86d00,FirstPage,NaN,test
1275,Tobacco800,ztz52d00-page02_1,FirstPage,NaN,test
1276,Tobacco800,ztz52d00-page02_2,NextPage,NaN,test
